In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np

from BReGNeXt import BReGNeXt as Classif
import cv2

cuda = torch.device('cuda:0')

In [2]:
classif = Classif()
tmp = torch.randn(10, 3, 224, 224)
classif(tmp).shape

torch.Size([10, 8])

In [3]:
from torch.utils.data import DataLoader

prepr = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                     std=[0.5, 0.5, 0.5])
])

import pickle as pkl
with open('data/affectnet/train/0', 'rb') as f:
    pack = pkl.load(f)
    
def gen_batch(bdata):
    new_width = 224
    new_height = 224
    inp = torch.zeros(len(bdata), 3, 224, 224)
    labels = torch.zeros(len(bdata), dtype=torch.long)
    for i, (im, label) in enumerate(bdata):
        im = cv2.resize(im, (new_width, new_height))
        inp[i] = prepr(im)
        labels[i] = label
    return inp.to(cuda), labels.to(cuda)

dloader = DataLoader(pack, batch_size=16, shuffle=True, collate_fn=gen_batch)

In [4]:
def focal_loss2(input_tensor, target_tensor, weight=None, gamma=2, reduction='mean'):
    log_prob = torch.nn.functional.log_softmax(input_tensor, dim=-1)
    probs = torch.exp(log_prob)
    return torch.nn.functional.nll_loss(((1 - probs) ** gamma) * log_prob,
                                        target_tensor, weight=weight, reduction=reduction)

In [5]:
optimizer = torch.optim.Adam(classif.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
# criterion = focal_loss2
classif = classif.to(cuda)

for epoch in range(20):
    for p_num in range(17):
        with open('data/affectnet/train/' + str(p_num), 'rb') as f:
            pack = pkl.load(f)
        dloader = DataLoader(pack, batch_size=32, shuffle=True, collate_fn=gen_batch)
        
        for i, (x, labels) in enumerate(dloader):
            pred = classif(x)

            optimizer.zero_grad()
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()

            print('Epoch {} Pack {} Loss {}'.format(epoch, 
                                                    p_num, 
                                                    loss.item()))

Epoch 0 Pack 0 Loss 2.099370002746582
Epoch 0 Pack 0 Loss 2.112152099609375
Epoch 0 Pack 0 Loss 2.0076189041137695
Epoch 0 Pack 0 Loss 2.1024363040924072
Epoch 0 Pack 0 Loss 2.0862271785736084
Epoch 0 Pack 0 Loss 2.1126251220703125
Epoch 0 Pack 0 Loss 2.1594066619873047
Epoch 0 Pack 0 Loss 2.1205813884735107
Epoch 0 Pack 0 Loss 2.156126022338867
Epoch 0 Pack 0 Loss 2.1378049850463867
Epoch 0 Pack 0 Loss 2.051867723464966
Epoch 0 Pack 0 Loss 2.1191248893737793
Epoch 0 Pack 0 Loss 2.1212525367736816
Epoch 0 Pack 0 Loss 2.0771350860595703
Epoch 0 Pack 0 Loss 2.1225080490112305
Epoch 0 Pack 0 Loss 2.0745491981506348
Epoch 0 Pack 0 Loss 2.073810577392578
Epoch 0 Pack 0 Loss 2.0684585571289062
Epoch 0 Pack 0 Loss 2.11226749420166
Epoch 0 Pack 0 Loss 2.046236038208008
Epoch 0 Pack 0 Loss 2.092975378036499
Epoch 0 Pack 0 Loss 2.064957618713379
Epoch 0 Pack 0 Loss 2.0750200748443604
Epoch 0 Pack 0 Loss 2.067272186279297
Epoch 0 Pack 0 Loss 2.122136354446411
Epoch 0 Pack 0 Loss 2.073954343795776

Epoch 0 Pack 6 Loss 2.0883588790893555
Epoch 0 Pack 6 Loss 2.074018716812134
Epoch 0 Pack 6 Loss 2.0344653129577637
Epoch 0 Pack 6 Loss 2.086704969406128
Epoch 0 Pack 6 Loss 2.085861921310425
Epoch 0 Pack 6 Loss 2.0791165828704834
Epoch 0 Pack 6 Loss 2.067791700363159
Epoch 0 Pack 6 Loss 2.113475799560547
Epoch 0 Pack 6 Loss 2.0175163745880127
Epoch 0 Pack 6 Loss 2.04394268989563
Epoch 0 Pack 6 Loss 2.028808116912842
Epoch 0 Pack 7 Loss 2.057652711868286
Epoch 0 Pack 7 Loss 2.1501717567443848
Epoch 0 Pack 7 Loss 2.134808301925659
Epoch 0 Pack 7 Loss 2.065831422805786
Epoch 0 Pack 7 Loss 2.0614945888519287
Epoch 0 Pack 7 Loss 2.082502841949463
Epoch 0 Pack 7 Loss 2.0625433921813965
Epoch 0 Pack 7 Loss 2.073364496231079
Epoch 0 Pack 7 Loss 2.0480127334594727
Epoch 0 Pack 7 Loss 2.038958787918091
Epoch 0 Pack 7 Loss 1.9820476770401
Epoch 0 Pack 7 Loss 2.06292462348938
Epoch 0 Pack 7 Loss 2.0620951652526855
Epoch 0 Pack 7 Loss 2.0946295261383057
Epoch 0 Pack 7 Loss 2.0641543865203857
Epoch

Epoch 0 Pack 13 Loss 2.020791530609131
Epoch 0 Pack 13 Loss 2.0891952514648438
Epoch 0 Pack 13 Loss 2.1183722019195557
Epoch 0 Pack 13 Loss 2.0422608852386475
Epoch 0 Pack 13 Loss 2.103459596633911
Epoch 0 Pack 13 Loss 2.060932159423828
Epoch 0 Pack 13 Loss 2.1022136211395264
Epoch 0 Pack 13 Loss 2.058276653289795
Epoch 0 Pack 13 Loss 2.0713260173797607
Epoch 0 Pack 13 Loss 2.1219053268432617
Epoch 0 Pack 13 Loss 2.0464184284210205
Epoch 0 Pack 13 Loss 2.0581352710723877
Epoch 0 Pack 13 Loss 2.119528293609619
Epoch 0 Pack 13 Loss 2.1060140132904053
Epoch 0 Pack 13 Loss 2.045548439025879
Epoch 0 Pack 13 Loss 2.041815996170044
Epoch 0 Pack 13 Loss 2.0766890048980713
Epoch 0 Pack 13 Loss 2.0402514934539795
Epoch 0 Pack 13 Loss 2.042949914932251
Epoch 0 Pack 13 Loss 2.0980868339538574
Epoch 0 Pack 13 Loss 2.0722718238830566
Epoch 0 Pack 13 Loss 2.074328899383545
Epoch 0 Pack 13 Loss 2.0806925296783447
Epoch 0 Pack 13 Loss 2.1595206260681152
Epoch 0 Pack 14 Loss 2.0748727321624756
Epoch 0 P

Epoch 1 Pack 2 Loss 2.110436201095581
Epoch 1 Pack 2 Loss 2.063718318939209
Epoch 1 Pack 2 Loss 2.042201042175293
Epoch 1 Pack 2 Loss 2.022557258605957
Epoch 1 Pack 2 Loss 2.06327223777771
Epoch 1 Pack 2 Loss 2.1084909439086914
Epoch 1 Pack 3 Loss 2.0172438621520996
Epoch 1 Pack 3 Loss 2.0535128116607666
Epoch 1 Pack 3 Loss 2.03855299949646
Epoch 1 Pack 3 Loss 2.0741798877716064
Epoch 1 Pack 3 Loss 2.1795589923858643
Epoch 1 Pack 3 Loss 2.0277860164642334
Epoch 1 Pack 3 Loss 2.056340217590332
Epoch 1 Pack 3 Loss 2.025651693344116
Epoch 1 Pack 3 Loss 2.0892443656921387
Epoch 1 Pack 3 Loss 2.0658459663391113
Epoch 1 Pack 3 Loss 2.10819935798645
Epoch 1 Pack 3 Loss 2.068620204925537
Epoch 1 Pack 3 Loss 2.1023387908935547
Epoch 1 Pack 3 Loss 2.097424268722534
Epoch 1 Pack 3 Loss 2.048286199569702
Epoch 1 Pack 3 Loss 2.1131627559661865
Epoch 1 Pack 3 Loss 2.1112148761749268
Epoch 1 Pack 3 Loss 2.019700765609741
Epoch 1 Pack 3 Loss 2.0550944805145264
Epoch 1 Pack 3 Loss 2.086881160736084
Epo

Epoch 1 Pack 9 Loss 2.0968973636627197
Epoch 1 Pack 9 Loss 2.1029324531555176
Epoch 1 Pack 9 Loss 2.0737264156341553
Epoch 1 Pack 9 Loss 2.0136923789978027
Epoch 1 Pack 9 Loss 2.1066815853118896
Epoch 1 Pack 9 Loss 2.1135947704315186
Epoch 1 Pack 9 Loss 2.062265157699585
Epoch 1 Pack 9 Loss 2.0891501903533936
Epoch 1 Pack 9 Loss 2.0235602855682373
Epoch 1 Pack 9 Loss 2.0813469886779785
Epoch 1 Pack 9 Loss 2.095081090927124
Epoch 1 Pack 9 Loss 2.0389740467071533
Epoch 1 Pack 9 Loss 1.992294192314148
Epoch 1 Pack 9 Loss 2.0740926265716553
Epoch 1 Pack 9 Loss 2.0851895809173584
Epoch 1 Pack 9 Loss 2.1068873405456543
Epoch 1 Pack 9 Loss 2.1002299785614014
Epoch 1 Pack 10 Loss 2.069237470626831
Epoch 1 Pack 10 Loss 2.115262508392334
Epoch 1 Pack 10 Loss 2.096195936203003
Epoch 1 Pack 10 Loss 2.0830492973327637
Epoch 1 Pack 10 Loss 2.1232316493988037
Epoch 1 Pack 10 Loss 1.9348398447036743
Epoch 1 Pack 10 Loss 1.9956027269363403
Epoch 1 Pack 10 Loss 2.0503928661346436
Epoch 1 Pack 10 Loss 2.

...

Epoch 6 Pack 15 Loss 1.8974864482879639
Epoch 6 Pack 15 Loss 1.9346212148666382
Epoch 6 Pack 15 Loss 2.08103346824646
Epoch 6 Pack 15 Loss 1.8689419031143188
Epoch 6 Pack 15 Loss 1.7647106647491455
Epoch 6 Pack 15 Loss 2.0728068351745605
Epoch 6 Pack 15 Loss 1.7189849615097046
Epoch 6 Pack 15 Loss 1.7815742492675781
Epoch 6 Pack 15 Loss 1.8517608642578125
Epoch 6 Pack 15 Loss 1.8656381368637085
Epoch 6 Pack 15 Loss 1.8425981998443604
Epoch 6 Pack 15 Loss 2.0332157611846924
Epoch 6 Pack 15 Loss 2.0848329067230225
Epoch 6 Pack 15 Loss 2.0219364166259766
Epoch 6 Pack 15 Loss 2.0766773223876953
Epoch 6 Pack 15 Loss 1.9823280572891235
Epoch 6 Pack 15 Loss 1.9456603527069092
Epoch 6 Pack 15 Loss 1.7295279502868652
Epoch 6 Pack 15 Loss 1.807992696762085
Epoch 6 Pack 15 Loss 1.8461518287658691
Epoch 6 Pack 15 Loss 1.9530001878738403
Epoch 6 Pack 15 Loss 1.8459460735321045
Epoch 6 Pack 15 Loss 1.959931492805481
Epoch 6 Pack 15 Loss 1.8527923822402954
Epoch 6 Pack 15 Loss 1.863635778427124
Epoch

Epoch 7 Pack 4 Loss 1.7184022665023804
Epoch 7 Pack 4 Loss 1.905843734741211
Epoch 7 Pack 4 Loss 1.9065301418304443
Epoch 7 Pack 4 Loss 1.8935195207595825
Epoch 7 Pack 4 Loss 1.894659399986267
Epoch 7 Pack 4 Loss 2.1102218627929688
Epoch 7 Pack 4 Loss 1.863330602645874
Epoch 7 Pack 4 Loss 2.0183632373809814
Epoch 7 Pack 4 Loss 1.663130760192871
Epoch 7 Pack 4 Loss 1.6247209310531616
Epoch 7 Pack 5 Loss 1.9545657634735107
Epoch 7 Pack 5 Loss 1.9126793146133423
Epoch 7 Pack 5 Loss 1.9284729957580566
Epoch 7 Pack 5 Loss 1.8670101165771484
Epoch 7 Pack 5 Loss 1.9336296319961548
Epoch 7 Pack 5 Loss 1.654174566268921
Epoch 7 Pack 5 Loss 1.9907127618789673
Epoch 7 Pack 5 Loss 1.9188669919967651
Epoch 7 Pack 5 Loss 1.861690640449524
Epoch 7 Pack 5 Loss 1.7937415838241577
Epoch 7 Pack 5 Loss 1.92982816696167
Epoch 7 Pack 5 Loss 1.692345142364502
Epoch 7 Pack 5 Loss 1.9406675100326538
Epoch 7 Pack 5 Loss 1.695752501487732
Epoch 7 Pack 5 Loss 1.9337666034698486
Epoch 7 Pack 5 Loss 1.7603046894073

Epoch 7 Pack 11 Loss 1.7294251918792725
Epoch 7 Pack 11 Loss 1.742273211479187
Epoch 7 Pack 11 Loss 1.767956018447876
Epoch 7 Pack 11 Loss 1.7340929508209229
Epoch 7 Pack 11 Loss 1.661218523979187
Epoch 7 Pack 11 Loss 1.763063669204712
Epoch 7 Pack 11 Loss 1.8478474617004395
Epoch 7 Pack 11 Loss 1.8308544158935547
Epoch 7 Pack 11 Loss 1.7436622381210327
Epoch 7 Pack 11 Loss 1.8335380554199219
Epoch 7 Pack 11 Loss 1.893988847732544
Epoch 7 Pack 11 Loss 1.9378116130828857
Epoch 7 Pack 11 Loss 1.897444486618042
Epoch 7 Pack 11 Loss 1.8396583795547485
Epoch 7 Pack 11 Loss 1.8567249774932861
Epoch 7 Pack 11 Loss 1.9545940160751343
Epoch 7 Pack 11 Loss 1.8787554502487183
Epoch 7 Pack 11 Loss 1.9628331661224365
Epoch 7 Pack 11 Loss 1.7638919353485107
Epoch 7 Pack 11 Loss 1.9394935369491577
Epoch 7 Pack 11 Loss 1.783259630203247
Epoch 7 Pack 11 Loss 1.8879834413528442
Epoch 7 Pack 11 Loss 1.9767906665802002
Epoch 7 Pack 12 Loss 1.653669834136963
Epoch 7 Pack 12 Loss 1.7602083683013916
Epoch 7 

Epoch 8 Pack 0 Loss 1.9118067026138306
Epoch 8 Pack 0 Loss 1.7084215879440308
Epoch 8 Pack 0 Loss 1.729546070098877
Epoch 8 Pack 0 Loss 1.871042251586914
Epoch 8 Pack 0 Loss 1.8933507204055786
Epoch 8 Pack 0 Loss 1.7271519899368286
Epoch 8 Pack 0 Loss 1.8422009944915771
Epoch 8 Pack 0 Loss 1.834275484085083
Epoch 8 Pack 1 Loss 2.0099000930786133
Epoch 8 Pack 1 Loss 1.9483681917190552
Epoch 8 Pack 1 Loss 1.8455862998962402
Epoch 8 Pack 1 Loss 1.837727427482605
Epoch 8 Pack 1 Loss 1.7462869882583618
Epoch 8 Pack 1 Loss 2.030820846557617
Epoch 8 Pack 1 Loss 1.7444465160369873
Epoch 8 Pack 1 Loss 1.9028204679489136
Epoch 8 Pack 1 Loss 1.6679540872573853
Epoch 8 Pack 1 Loss 1.7056963443756104
Epoch 8 Pack 1 Loss 1.8067487478256226
Epoch 8 Pack 1 Loss 2.0168683528900146
Epoch 8 Pack 1 Loss 1.7195215225219727
Epoch 8 Pack 1 Loss 1.9115259647369385
Epoch 8 Pack 1 Loss 1.926081657409668
Epoch 8 Pack 1 Loss 1.7578407526016235
Epoch 8 Pack 1 Loss 1.8321943283081055
Epoch 8 Pack 1 Loss 1.765856266

KeyboardInterrupt: 

In [6]:
torch.save(classif.state_dict(), 'graphs/bregxnet')

In [7]:
hits = 0
total = 0
for p_num in range(3):
    with open('data/affectnet/val/' + str(p_num), 'rb') as f:
        pack = pkl.load(f)
    dloader = DataLoader(pack, batch_size=8, collate_fn=gen_batch)
    
    for i, (x, labels) in enumerate(dloader):
        pred = classif(x)
        
        pr_labels = torch.argmax(pred.detach(), dim=-1)
        hits += (labels == torch.argmax(pred, dim=-1)).sum()
        total += labels.shape[0]
        
print('Accuracy = {}'.format(hits / total))

Accuracy = 0.289000004529953
